In [10]:
from claim_modelling_kedro.pipelines.utils.dataframes import (
    load_metrics_table_from_mlflow,
    load_predictions_and_target_from_mlflow,
    load_metrics_cv_stats_from_mlflow
)
from claim_modelling_kedro.pipelines.utils.datasets import (
    get_partition,
    get_mlflow_run_id_for_partition,
)
from claim_modelling_kedro.experiments.experiment import create_experiment_run, default_run_name_from_run_no, get_run_mlflow_id

In [1]:
import sys
sys.path.insert(0, "/home/krzysiek/Development/claim-modelling/claim_modelling_kedro/notebooks/utils")

from utils.my_kedro import MyKedro, suppress_all_loggers
import logging

logger = logging.getLogger(__name__)
logger.setLevel("INFO")

my_kedro = MyKedro()

[05/30/25 15:37:43] INFO                    context.py:L91       mlflow_config.server.mlflow_tracking_uri:         
                             http://127.0.0.1:5000

                    INFO                    context.py:L106      mlflow_run_id: a220ddaad3744be9ba3b799066ddbc5a

                    INFO                   __init__.py:L134      Kedro project solvency-models

                    INFO                   __init__.py:L135      Defined global variable 'context', 'session',     
                             'catalog' and 'pipelines'

                    INFO                   my_kedro.py:L125      Reloaded kedro. Active mlflow_run_id:             
                             a220ddaad3744be9ba3b799066ddbc5a

In [2]:
my_kedro.reload()

[05/30/25 15:37:44] INFO                    context.py:L91       mlflow_config.server.mlflow_tracking_uri:         
                             http://127.0.0.1:5000

                    INFO                    context.py:L106      mlflow_run_id: a220ddaad3744be9ba3b799066ddbc5a

[05/30/25 15:37:45] INFO                   __init__.py:L134      Kedro project solvency-models

                    INFO                   __init__.py:L135      Defined global variable 'context', 'session',     
                             'catalog' and 'pipelines'

                    INFO                   my_kedro.py:L125      Reloaded kedro. Active mlflow_run_id:             
                             a220ddaad3744be9ba3b799066ddbc5a

In [4]:
my_kedro.run("init", level="ERROR")
config = catalog.load("config")

{}

## Dummy mean regressor

In [11]:
dummy_exp_name = "sev_001_dummy_mean_regressor"
dummy_run_name = "dummy_mean"
dummy_run_id = get_run_mlflow_id(dummy_exp_name, dummy_run_name)

[05/30/25 18:33:39] INFO                 experiment.py:L185      MLflow Run ID for sev_001_dummy_mean_regressor,   
                             Run dummy_mean: 5c5c70ecd2ec491ba8b4e7f6552dc753

In [32]:
test_dummy_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="test", mlflow_run_id=dummy_run_id)
train_dummy_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="sample_train_pure", mlflow_run_id=dummy_run_id)

In [30]:
test_dummy_cv_stats.round(3)

,cv_mean,cv_std
metric,,
test_nwMGD,1.493,0.046
test_nwMAE,43.077,0.583
test_nwRMSE,9022.634,678.876
test_nwR2,-0.000,0.000
test_nwMBD,-1.502,50.147
test_nwSC,-0.000,0.000
test_nwABC,0.000,0.000
test_nwCCI,0.024,0.008
test_nwCOI,0.017,0.010


In [33]:
train_dummy_cv_stats.round(3)

,cv_mean,cv_std
metric,,
sample_train_pure_nwMGD,1.495,0.007
sample_train_pure_nwMAE,43.103,0.161
sample_train_pure_nwRMSE,9001.243,218.032
sample_train_pure_nwR2,0.000,0.000
sample_train_pure_nwMBD,0.000,0.000
sample_train_pure_nwSC,-0.000,0.000
sample_train_pure_nwABC,0.000,0.000
sample_train_pure_nwCCI,0.016,0.002
sample_train_pure_nwCOI,0.012,0.004


## Statsmodels GLM

### Wuthrich baseline

In [37]:
glm_de_exp_name = "sev_002_statsmodels_glm_de"
baseline_run_name = "wuthrich_baseline"
baseline_run_id = get_run_mlflow_id(glm_de_exp_name, baseline_run_name)
test_wuthrich_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="test", mlflow_run_id=baseline_run_id)
train_wuthrich_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="sample_train_pure", mlflow_run_id=baseline_run_id)

[05/30/25 19:45:09] INFO                 experiment.py:L185      MLflow Run ID for sev_002_statsmodels_glm_de, Run 
                             wuthrich_baseline: 44f7611ffc4247c8b492ba700d2dc7d8

In [38]:
test_wuthrich_cv_stats.round(3)

,cv_mean,cv_std
metric,,
test_nwMGD,188.970,79.295
test_nwMAE,61.502,4.257
test_nwRMSE,35095.564,28735.109
test_nwR2,-23.844,41.886
test_nwMBD,-1732.854,560.853
test_nwSC,-0.001,0.015
test_nwABC,0.319,0.028
test_nwCCI,0.320,0.024
test_nwCOI,0.000,0.000


In [39]:
train_wuthrich_cv_stats.round(3)

,cv_mean,cv_std
metric,,
sample_train_pure_nwMGD,209.050,66.377
sample_train_pure_nwMAE,61.844,11.877
sample_train_pure_nwRMSE,71853.379,122585.145
sample_train_pure_nwR2,-209.522,462.541
sample_train_pure_nwMBD,-1972.623,1649.688
sample_train_pure_nwSC,0.011,0.010
sample_train_pure_nwABC,0.247,0.045
sample_train_pure_nwCCI,0.263,0.064
sample_train_pure_nwCOI,0.000,0.000


In [48]:
glm_de_exp_name = "sev_002_statsmodels_glm_de"
run_name = "wuthrich_standard_scaler_mean "
run_id = get_run_mlflow_id(glm_de_exp_name, run_name)
test_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="test", mlflow_run_id=run_id)
train_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="sample_train_pure", mlflow_run_id=run_id)

[05/30/25 23:20:17] INFO                 experiment.py:L188      MLflow run ID file                                
                             experiments/sev_002_statsmodels_glm_de/runs/wuthrich_standard_scaler_mean             
                             /mlflow_run_id.txt not found.

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 glm_de_exp_name = "sev_002_statsmodels_glm_de"                                               │
│   2 run_name = "wuthrich_standard_scaler_mean "                                                  │
│   3 run_id = get_run_mlflow_id(glm_de_exp_name, run_name)                                        │
│ ❱ 4 test_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="test", mlflow_run_id=run_id)      │
│   5 train_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="sample_train_pure", mlflow_r     │
│   6                                                                                              │
│                                                                                                  │
│ /home/krzysiek/Development/claim-modelling-kedro/claim_modelling_kedro/src/claim_modelling_kedro │
│ /pipelines/utils/dataframes.py:192 in load_metrics_cv_stats_from_mlflow                          │
│                                                                                                  │
│   189                                                                                            │
│   190 def load_metrics_cv_stats_from_mlflow(dataset: str, prefix: str = None, mlflow_run_id: s   │
│   191 │   filename, artifact_path = get_metrics_cv_stats_filname_and_artifact_path(dataset, pr   │
│ ❱ 192 │   metrics_cv_stats_df = load_pd_dataframe_csv_from_mlflow(artifact_path, filename, mlf   │
│   193 │   return metrics_cv_stats_df                                                             │
│   194                                                                                            │
│                                                                                                  │
│ /home/krzysiek/Development/claim-modelling-kedro/claim_modelling_kedro/src/claim_modelling_kedro │
│ /pipelines/utils/dataframes.py:149 in load_pd_dataframe_csv_from_mlflow                          │
│                                                                                                  │
│   146 │   with tempfile.TemporaryDirectory() as temp_dir:                                        │
│   147 │   │   artifact_path = f"{artifact_path}/{filename}" if filename is not None else artif   │
│   148 │   │   artifact_uri = f"runs:/{mlflow_run_id}/{artifact_path}"                            │
│ ❱ 149 │   │   local_path = mlflow.tracking.artifact_utils._download_artifact_from_uri(artifact   │
│   150 │   │   df = pd.read_csv(local_path, index_col=index_col)                                  │
│   151 │   │   df.columns.name = columns_index_name                                               │
│   152 │   │   return df                                                                          │
│                                                                                                  │
│ /home/krzysiek/Development/claim-modelling-kedro/venv/lib/python3.11/site-packages/mlflow/tracki │
│ ng/artifact_utils.py:105 in _download_artifact_from_uri                                          │
│                                                                                                  │
│   102 │   │   │   a local output path will be created.                                           │
│   103 │   """                                                                                    │
│   104 │   root_uri, artifact_path = _get_root_uri_and_artifact_path(artifact_uri)                │
│ ❱ 105 │   return get_artifact_repository(artifact_uri=root_uri).download_artifacts(              │
│   106 │   │   artifact_path=artifact_path, dst_path=output_path                                  │
│   107 │   )                                                

In [ ]:
test_cv_stats.round(3)

### Categorical area

In [44]:
glm_de_exp_name = "sev_002_statsmodels_glm_de"
run_name = "cat_area_baseline"
run_id = get_run_mlflow_id(glm_de_exp_name, run_name)
test_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="test", mlflow_run_id=run_id)
train_cv_stats = load_metrics_cv_stats_from_mlflow(dataset="sample_train_pure", mlflow_run_id=run_id)

[05/30/25 20:30:10] INFO                 experiment.py:L185      MLflow Run ID for sev_002_statsmodels_glm_de, Run 
                             cat_area_baseline: be8fb278e1e44bb6a9883b1495770cbb

In [45]:
test_cv_stats.round(3)

,cv_mean,cv_std
metric,,
test_nwMGD,169.770,76.630
test_nwMAE,69.253,12.054
test_nwRMSE,100667.359,96113.575
test_nwR2,-208.537,287.733
test_nwMBD,-2849.614,1698.534
test_nwSC,0.005,0.005
test_nwABC,0.345,0.049
test_nwCCI,0.351,0.044
test_nwCOI,0.000,0.000


In [47]:
test_wuthrich_cv_stats.round(3)

,cv_mean,cv_std
metric,,
test_nwMGD,188.970,79.295
test_nwMAE,61.502,4.257
test_nwRMSE,35095.564,28735.109
test_nwR2,-23.844,41.886
test_nwMBD,-1732.854,560.853
test_nwSC,-0.001,0.015
test_nwABC,0.319,0.028
test_nwCCI,0.320,0.024
test_nwCOI,0.000,0.000
